In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
maxitem = requests.get("https://hacker-news.firebaseio.com/v0/maxitem.json?print=pretty").json()

In [4]:
import sqlalchemy
from decouple import config 
import psycopg2
host = config('DATABASE_URL')
db = sqlalchemy.create_engine(host)

In [21]:
conn = db.connect()
curs = conn.connection.cursor()

In [23]:
curs.execute(" SELECT Count(id) FROM comments")
count = curs.fetchall()

In [6]:
curs.execute(" SELECT Max(id) FROM comments")
tmax = curs.fetchall()

In [7]:
maxitem - tmax[0][0]

41157

In [8]:
def hn_scrape(i):
    r = requests.get('https://hacker-news.firebaseio.com/v0/item/'+str(i)+'.json').json()
    try:
        if ('deleted' in r.keys()):
            pass
        else:
            if r["type"] == 'comment':
                t = (r["by"],r["id"],r["text"],r["time"])
                comments_list.append(t)
    except:
        pass
                      

In [9]:
comments_list = []

In [ ]:
for i in range(tmax[0][0],maxitem):
    hn_scrape(i)

In [ ]:
comments= pd.DataFrame(columns = ["by", "id","text","time"], data=comments_list)

In [ ]:
comments["text"] = comments["text"].str.replace("&quot;","")
comments["text"] = comments["text"].str.replace("&#x27;","'")
comments["text"] = comments["text"].str.replace("&gt; "," ")
comments["text"] = comments["text"].str.replace("<p>"," ")
comments["text"] = comments["text"].str.replace("<a>"," ")
comments["text"] = comments["text"].str.replace("</a>"," ")
comments["text"] = comments["text"].str.replace("<i>"," ")
comments["text"] = comments["text"].str.replace("</i>"," ")
comments["text"] = comments["text"].str.replace("&#x2F;",'')
comments["text"] = comments["text"].str.replace("https:",' ')
comments["text"] = comments["text"].str.replace("\\n",' ')

In [ ]:
comments.tail()

In [ ]:
def sentiment(text):
    payload = {'text':text}
    try:
        return requests.get('https://crawftv-nlp-api.herokuapp.com/sentiment',params=payload).json()["compound"]
    except:
        pass

In [ ]:
comments["sentiment"] = comments["text"].apply(sentiment)

In [ ]:
comments = comments.dropna()

In [ ]:
comments.to_sql(name="comments",con=db, if_exists="append",chunksize=500)

In [ ]:
conn.close()
curs.close()